# SRoBERTa의 전이학습을 활용한 보고서 원인 추출 모델 개발

In [1]:
HUGGINGFACE_MODEL_PATH = 'jhgan/ko-sroberta-multitask'
GPU_NUM = 0
SEED = 42

MODEL_PATH = f'{HUGGINGFACE_MODEL_PATH.replace("/", "-")}/{HUGGINGFACE_MODEL_PATH.replace("/", "-")}.pt'

num_epochs = 200
BATCH_SIZE = 512

MIN_NGRAM = 3
MAX_NGRAM = 8

SEQUENCE_MATCHER_THRESHOLD = 0.6

## (0) 기본 셋팅

In [2]:
import os
import torch
import random

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= f"{GPU_NUM}"  # Set the GPU number to use

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda
Current cuda device: 0
Count of using GPUs: 1


In [3]:
from sentence_transformers import SentenceTransformer

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import pandas as pd
import numpy as np
from tqdm import tqdm
import re
import pickle

In [4]:
# 추론 및 평가를 위한 라이브러리
from sklearn.feature_extraction.text import CountVectorizer # CountVectorizer를 사용하는 이유는 n_gram_range의 인자를 사용하면 단쉽게 n-gram을 추출할 수 있기 때문입니다. 
from sklearn.metrics.pairwise import cosine_similarity
from difflib import SequenceMatcher

def similar(predict, ground):
    return SequenceMatcher(None, predict, ground).ratio()

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if device =='cuda':
        torch.cuda.manual_seed_all(seed)

seed_everything(SEED) # Seed 고정

### 사전학습 모델 불러오기

In [6]:
SBERT_model=SentenceTransformer(HUGGINGFACE_MODEL_PATH, device=device)

## (1) 학습 데이터 준비

### (1-1) 훈련 데이터셋 불러오기

In [7]:
train_df = pd.read_csv("../data/rawdata/GYRO_trainset.csv")

In [8]:
train_df.tail()

,idx,근거,사고명,본문,원인 키워드,원인,발생지표,요약,재발방지대책,조치결과,DATA,비고
869,398.00,GYRO 92호,접근 중 절차고도 미준수,"인천공항에 착륙을 위해 고도 약 12,000ft에서 강하 접근 중 Seoul App...","""300ft가 낮은 4,800ft로 통과하였음""",O 발견동기:FIX 통과 직전에 고도계를 확인하는 과정에서 발견. \nO 발생요인:...,NaN,NaN,NaN,NaN,NaN,NaN
870,550.00,GYRO 110호,APU FIRE MESSAGE 시현 관련 정비조치 사례,zz공항에서 비행을 준비 중 FIRE LOOP 1 APU START MASSAGE가...,"""FIRE LOOP 1 APU START MASSAGE가 시현되어"", ""FIRE A...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
871,166.00,GYRO 39호,TAIL-SKID 마모,◆ 경위 \n제주 Runway24 ILS로 착륙 시(바람: 320/22) 고도 10...,"""Nose up 되는 것을 인지하고 막는 순간 Action이 늦었음."", ""Powe...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
872,"1,185.00",GYRO 126,NaN,홍콩공항에서 RWY 07R로 이륙하기 위하여 TAXIWAY ON 'K'에서 TWR로...,"""'K1까지 TAXI 하라'는 것으로 오인하여 'K1'까지 진입하였음""",NaN,NaN,NaN,NaN,NaN,NaN,NaN
873,984.00,GYRO 146,NaN,"짧은 비행이었지만, 난기류가 심한 날씨였다. 순항고도에 이르자 기장은 좌석벨트 지시...","""기장이나와 뒤편에 위치해있던 객실승무원의 보고에 귀를 기울이지 않았다는 것이다.""",NaN,NaN,NaN,NaN,NaN,NaN,NaN


### (1-2) `train_df`를 **페어 데이터셋**으로 변환하기
- 페어 데이터셋의 구성
    - '본문' - 'phrase' - '이진 라벨'
        - '이진 라벨'의 의미 : 'phrase'가 '본문'의 원인인지 아닌지 여부

In [9]:
def process_doc(doc):
    processed_doc = doc.strip('"').replace("\n", " ").strip()
    processed_doc = re.sub('\s+', ' ', processed_doc)
    
    return processed_doc

def tokenizer(string):
    return string.split(" ")

def doc_to_candidate_phrase(doc, n_gram_range=(MIN_NGRAM, MAX_NGRAM)):
    # 보고서를 입력으로 받으면,
    # 보고서에서 후보 phrase를 추출하여 반환하는 함수
    
    processed_doc = process_doc(doc)
    tokenized_doc = " ".join(processed_doc.split(' '))
    
    count = CountVectorizer(ngram_range=n_gram_range, lowercase=False, tokenizer=tokenizer, token_pattern=None).fit([tokenized_doc])
    candidates = count.get_feature_names_out()
    
    return candidates

In [10]:
doc_to_candidate_phrase(train_df["본문"][0])

array(["'MAYDAY, MAYDAY, MAYDAY.", "'MAYDAY, MAYDAY, MAYDAY. ABC",
       "'MAYDAY, MAYDAY, MAYDAY. ABC 유도로에서", ...,
       '힘으로 인해 의자 사이로 넘어졌다. 그러자', '힘으로 인해 의자 사이로 넘어졌다. 그러자 그는',
       '힘으로 인해 의자 사이로 넘어졌다. 그러자 그는 내'], dtype=object)

In [11]:
def make_pair_dataset(df):
    pair_dataset = []
    for i in tqdm(range(len(df))):
        doc = df["본문"][i]
        candidates = doc_to_candidate_phrase(doc)    
        gold_standards = df["원인 키워드"][i]

        for candidate_phrase in candidates:
            binary_label = 0
            for gold_standard in gold_standards:
                # match 기법 1
                # candidate_phrase가 gold_standard와 정확히 일치할 때만 인정
                if candidate_phrase == gold_standard: 
                    binary_label = 1
                    break

                # match 기법 1
                # candidate_phrase가 gold_standard와 정확히 일치할 때만 인정
                elif similar(candidate_phrase, gold_standard) >= SEQUENCE_MATCHER_THRESHOLD:
                    binary_label = 1
                    break

            pair_dataset.append({'i': i, 'doc_idx': df["idx"][i], 'candidate': candidate_phrase, 'binary_label': binary_label})
            
    pair_dataset_df = pd.DataFrame(data=pair_dataset,
                              columns=['i', 'doc_idx', 'candidate', 'binary_label'])
    
    return pair_dataset_df

In [12]:
MODE = "train"

In [13]:
if os.path.isfile(f'../data/pair_dataset/{MODE}_pair_dataset_df_{str(SEQUENCE_MATCHER_THRESHOLD).replace(".", "_")}.pkl'):
    with open(f'../data/pair_dataset/{MODE}_pair_dataset_df_{str(SEQUENCE_MATCHER_THRESHOLD).replace(".", "_")}.pkl', 'rb') as f:
        train_pair_dataset_df = pickle.load(f)
else:
    train_pair_dataset_df = make_pair_dataset(train_df)
    with open(f'../data/pair_dataset/{MODE}_pair_dataset_df_{str(SEQUENCE_MATCHER_THRESHOLD).replace(".", "_")}.pkl', 'wb') as f:
        pickle.dump(train_pair_dataset_df, f)

In [14]:
train_pair_dataset_df

,i,doc_idx,candidate,binary_label
0,0,7.00,"'MAYDAY,",0
1,0,7.00,"'MAYDAY, MAYDAY,",0
2,0,7.00,"'MAYDAY, MAYDAY, MAYDAY.",0
3,0,7.00,"'MAYDAY, MAYDAY, MAYDAY. ABC",0
4,0,7.00,"'MAYDAY, MAYDAY, MAYDAY. ABC 유도로에서",0
...,...,...,...,...
1038646,873,984.00,했다. 내가 말하고 싶은 것은 기장이나와,0
1038647,873,984.00,했다. 내가 말하고 싶은 것은 기장이나와 뒤편에,0
1038648,873,984.00,했다. 내가 말하고 싶은 것은 기장이나와 뒤편에 위치해있던,0
1038649,873,984.00,했다. 내가 말하고 싶은 것은 기장이나와 뒤편에 위치해있던 객실승무원의,0


In [15]:
sum(train_pair_dataset_df["binary_label"])

12569

## (2) 데이터 전처리

### (2-1) (사전학습 모델을 통한) Input Document 임베딩 및 Candidate Phrase 임베딩

In [16]:
MODE = "train"

In [17]:
if os.path.isfile(f'../embeddings/{MODE}/{MODE}_doc_embedding_list.pkl'):
    with open(f'../embeddings/{MODE}/{MODE}_doc_embedding_list.pkl', 'rb') as f:
        doc_embedding_list = pickle.load(f)
else:
    doc_embedding_list = [SBERT_model.encode([doc]) for doc in tqdm(train_df["본문"])] # (482, 1, 768)    
    with open(f'../embeddings/{MODE}/{MODE}_doc_embedding_list.pkl', 'wb') as f:
        pickle.dump(doc_embedding_list, f)

In [18]:
if os.path.isfile(f'../embeddings/{MODE}/{MODE}_phr_embedding_list_{str(SEQUENCE_MATCHER_THRESHOLD).replace(".", "_")}.pkl'):
    with open(f'../embeddings/{MODE}/{MODE}_phr_embedding_list_{str(SEQUENCE_MATCHER_THRESHOLD).replace(".", "_")}.pkl', 'rb') as f:
        phr_embedding_list = pickle.load(f)
else:
    phr_embedding_list = [SBERT_model.encode([doc]) for doc in tqdm(train_pair_dataset_df["candidate"])] # (1038651, 1, 768)
    with open(f'../embeddings/{MODE}/{MODE}_phr_embedding_list_{str(SEQUENCE_MATCHER_THRESHOLD).replace(".", "_")}.pkl', 'wb') as f:
        pickle.dump(phr_embedding_list, f)

100% 1038651/1038651 [5:38:21<00:00, 51.16it/s] 


### (2-2) Embedding Vector들을 Concatenate

In [19]:
len(doc_embedding_list), len(phr_embedding_list)

(874, 1038651)

In [20]:
X_data = []
y_data = []

for idx in tqdm(range(len(train_pair_dataset_df))):
    concatenated_embedding = np.concatenate((doc_embedding_list[train_pair_dataset_df["i"][idx]], phr_embedding_list[idx]), axis=1)
    X_data.append(concatenated_embedding)
    y_data.append(train_pair_dataset_df["binary_label"][idx])

100% 1038651/1038651 [00:31<00:00, 33000.78it/s]


In [21]:
X_data = torch.tensor(X_data, dtype=torch.float32) # torch.Size([228898, 1, 1536])
y_data = torch.tensor(y_data, dtype=torch.float32) # torch.Size([228898])

/tmp/ipykernel_193668/2908007466.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  X_data = torch.tensor(X_data, dtype=torch.float32) # torch.Size([228898, 1, 1536])


### (2-3) Split Train & Validation Data 

In [22]:
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, shuffle=False, random_state=SEED)

In [23]:
sum(y_train).item(), sum(y_val).item()

(9989.0, 2580.0)

In [24]:
len(X_train), len(X_val)

(830920, 207731)

In [25]:
X_train, X_val, y_train, y_val = X_train.to(device), X_val.to(device), y_train.to(device), y_val.to(device)

### (2-4) dataset 및 dataloader 정의

In [26]:
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

## (3) Model - Linear Layer (for Binary Classification)

In [31]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.flatten = nn.Flatten()
        
        self.fc1 = nn.Linear(1536, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.fc4 = nn.Linear(128, 1)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.flatten(x)
        x = self.relu(self.bn1(self.fc1(x)))
        x = self.relu(self.bn2(self.fc2(x)))
        x = self.relu(self.bn3(self.fc3(x)))
        logit = self.fc4(x)
        
        return torch.sigmoid(logit)

In [32]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=1536, out_features=512, bias=True)
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (bn3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=128, out_features=1, bias=True)
  (relu): ReLU()
)


## (4) 모델 학습

In [33]:
criterion = nn.BCELoss()
# criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=0.0001)

In [34]:
train_loss_values = []  # To store training loss for each epoch
val_accuracy_values = []  # To store validation accuracy for each epoch
val_probabilities = []  # To store probabilities of the positive class for each validation sample

best_val_recall = 0.0
patience = 30  # Number of epochs to wait before early stopping
no_improvement_count = 0

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    
    # Training loop
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        
        # H(x) 계산
        logits = model(inputs)

        # loss 계산
        loss = criterion(logits.view(-1), labels)

        # loss로 H(x) 계산
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    # Compute training accuracy for the epoch
    with torch.no_grad():
        model.eval()  # Set the model to evaluation mode
        train_outputs = model(X_train)
        train_predictions = (train_outputs >= 0.5).long().view(-1).cpu()
        train_accuracy = accuracy_score(y_train.cpu(), train_predictions)
        train_precision = precision_score(y_train.cpu(), train_predictions)
        train_recall = recall_score(y_train.cpu(), train_predictions)
        train_f1 = f1_score(y_train.cpu(), train_predictions)
        train_loss_values.append(total_loss / len(train_loader))

        # Validation loop for computing validation accuracy, precision, recall, and F1-score
        val_outputs = model(X_val)
        val_predictions = (val_outputs >= 0.5).long().view(-1).cpu()
        val_accuracy = accuracy_score(y_val.cpu(), val_predictions)
        val_precision = precision_score(y_val.cpu(), val_predictions)
        val_recall = recall_score(y_val.cpu(), val_predictions)
        val_f1 = f1_score(y_val.cpu(), val_predictions)
        val_accuracy_values.append(val_accuracy)

        # Save the model with the best validation recall
        if val_recall > best_val_recall:
            best_val_recall = val_recall
            torch.save(model.state_dict(), MODEL_PATH)
            no_improvement_count = 0
        else:
            no_improvement_count += 1
            
        # Early stopping
        if no_improvement_count >= patience:
            print("Early stopping triggered.")
            break
            
        # Calculate probabilities of the positive class (class 1) for each validation sample
        val_probabilities.append(val_outputs.squeeze().tolist())

        print(f"Epoch {epoch+1}/{num_epochs}\n"
              f"Training Loss: {train_loss_values[-1]:.4f}, "
              f"Training Accuracy: {train_accuracy:.4f}, "
              f"Training Precision: {train_precision:.4f}, "
              f"Training Recall: {train_recall:.4f}, "
              f"Training F1-score: {train_f1:.4f}\n"
              f"Validation Accuracy: {val_accuracy:.4f}, "
              f"Validation Precision: {val_precision:.4f}, "
              f"Validation Recall: {val_recall:.4f}, "
              f"Validation F1-score: {val_f1:.4f}")

Epoch 1/200
Training Loss: 0.0324, Training Accuracy: 0.9914, Training Precision: 0.6602, Training Recall: 0.5807, Training F1-score: 0.6179
Validation Accuracy: 0.9900, Validation Precision: 0.6171, Validation Recall: 0.5128, Validation F1-score: 0.5601
Epoch 2/200
Training Loss: 0.0183, Training Accuracy: 0.9910, Training Precision: 0.6394, Training Recall: 0.5835, Training F1-score: 0.6102
Validation Accuracy: 0.9895, Validation Precision: 0.5892, Validation Recall: 0.5182, Validation F1-score: 0.5515
Epoch 3/200
Training Loss: 0.0164, Training Accuracy: 0.9935, Training Precision: 0.7301, Training Recall: 0.7302, Training F1-score: 0.7301
Validation Accuracy: 0.9908, Validation Precision: 0.6387, Validation Recall: 0.6023, Validation F1-score: 0.6200
Epoch 4/200
Training Loss: 0.0150, Training Accuracy: 0.9944, Training Precision: 0.7812, Training Recall: 0.7381, Training F1-score: 0.7590
Validation Accuracy: 0.9913, Validation Precision: 0.6714, Validation Recall: 0.5829, Validati

## (5) 모델 저장 및 불러오기

In [35]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.flatten = nn.Flatten()
        
        self.fc1 = nn.Linear(1536, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.fc4 = nn.Linear(128, 1)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.flatten(x)
        x = self.relu(self.bn1(self.fc1(x)))
        x = self.relu(self.bn2(self.fc2(x)))
        x = self.relu(self.bn3(self.fc3(x)))
        logit = self.fc4(x)
        
        return torch.sigmoid(logit)

In [36]:
torch.save(model.state_dict(), MODEL_PATH)

In [37]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load(MODEL_PATH))
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=1536, out_features=512, bias=True)
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (bn3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=128, out_features=1, bias=True)
  (relu): ReLU()
)


## (6) 모델 추론

In [38]:
n_gram_range = (MIN_NGRAM, MAX_NGRAM)
n_gram_range

(3, 8)

In [39]:
def process_doc(doc):
    processed_doc = doc.strip('"').replace("\n", " ").strip()
    processed_doc = re.sub('\s+', ' ', processed_doc)
    
    return processed_doc

def tokenizer(string):
    return string.split(" ")

def inference(doc_text, return_prob=False, fail_return=False):
    """
    doc_text (str) : 추론할 보고서
    return_prob (bool) : 예측 확률까지 같이 return 할지에 대한 여부
    fail_return (bool) : 원인 분석에 실패했을때, 그나마 가능성 있는 top5를 return 할지에 대한 여부
    """
    
    processed_doc = process_doc(doc_text)
    tokenized_doc = " ".join(processed_doc.split(' '))
    
    count = CountVectorizer(ngram_range=n_gram_range, lowercase=False, tokenizer=tokenizer, token_pattern=None).fit([tokenized_doc])    
    candidates = count.get_feature_names_out()
    
    test_doc_embedding = SBERT_model.encode([doc_text])
    test_phr_embedding_list = [SBERT_model.encode([phr]) for phr in (candidates)]
    
    X_test = []
    for test_phr_embedding in (test_phr_embedding_list):
        concatenated_embedding = np.concatenate((test_doc_embedding, test_phr_embedding), axis=1)
        X_test.append(concatenated_embedding)
    X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
    
    test_probabilities = []  # To store probabilities of the positive class for each test sample
    with torch.no_grad():
        model.eval()  # Set the model to evaluation mode

        # Test loop for computing test_predictions & test_probabilities
        test_outputs = model(X_test)
        test_predictions = (test_outputs >= 0.5).long().view(-1).cpu()
        test_probabilities.append(test_outputs.squeeze().tolist()) # Calculate probabilities of the positive class (class 1) for each test sample
        
    # Convert probabilities tensor to a numpy array
    probabilities = np.array(test_probabilities)[0]

    # Filter candidates with probabilities greater than or equal to 0.5
    filtered_candidates = candidates[probabilities >= 0.5]
    filtered_probabilities = probabilities[probabilities >= 0.5]

    # Sort candidates in descending order based on probabilities
    sorted_indices = np.argsort(filtered_probabilities)[::-1]
    sorted_candidates = filtered_candidates[sorted_indices]
    sorted_probabilities = filtered_probabilities[sorted_indices]

    
    if return_prob==True:
        predicted_phrases_and_prob_list = []

        # Check if there are candidates with probabilities greater than or equal to 0.5
        if len(sorted_candidates) > 0:
            # Print the sorted candidates and their corresponding probabilities
            for candidate, probability in zip(sorted_candidates, sorted_probabilities):
                predicted_phrases_and_prob_list.append( (candidate, probability) )
                # print(f"Predicted Keyphrase: '{candidate}', Probability: {probability:.6f}")
        else:
            if fail_return == True:
                predicted_phrases_and_prob_list.append( ("KEYPHRASE_PREDICTION_FAILED!", 0) )

                # If no candidates with probabilities >= 0.5, print top 5 candidates with the highest probabilities
                top_5_indices = np.argsort(probabilities)[::-1][:5]
                top_5_candidates = candidates[top_5_indices]
                top_5_probabilities = probabilities[top_5_indices]

                for candidate, probability in zip(top_5_candidates, top_5_probabilities):
                    predicted_phrases_and_prob_list.append( (candidate, probability) )
                    # print(f"Top-5 Candidate: '{candidate}', Probability: {probability:.6f}")

        return predicted_phrases_and_prob_list
    
    else:
        predicted_phrases_list = []

        # Check if there are candidates with probabilities greater than or equal to 0.5
        if len(sorted_candidates) > 0:
            # Print the sorted candidates and their corresponding probabilities
            for candidate, probability in zip(sorted_candidates, sorted_probabilities):
                predicted_phrases_list.append( candidate )
                # print(f"Predicted Keyphrase: '{candidate}', Probability: {probability:.6f}")
        else:
            if fail_return == True:
                predicted_phrases_list.append( ("KEYPHRASE_PREDICTION_FAILED!", 0) )

                # If no candidates with probabilities >= 0.5, print top 5 candidates with the highest probabilities
                top_5_indices = np.argsort(probabilities)[::-1][:5]
                top_5_candidates = candidates[top_5_indices]
                top_5_probabilities = probabilities[top_5_indices]

                for candidate, probability in zip(top_5_candidates, top_5_probabilities):
                    predicted_phrases_list.append( candidate )
                    # print(f"Top-5 Candidate: '{candidate}', Probability: {probability:.6f}")

        return predicted_phrases_list

In [40]:
doc_text = """- 00비행장에서 IFR(Instrument Flight Rules, 계기비행 규칙) 비행으로 초기 상승 중이었던 항공기 A에 대해, 주변의 최저 유도 고도*(MVA: Minimum Vectoring Altitude)를 혼동하여 당시 고도 약 4,000ft에 있던 항공기를 해당 MVA (약 5,000ft) 미만에서 레이더 유도하였던 내용에 대한 자율 보고서임.
* 최저 유도 고도(MVA): 항공교통 관제 기관이 안전한 레이더 관제를 실시하는 데 활용하기 위해 설정한 고도로, 가장 높은 장애물로부터 1,000ft의 분리(비산악 지역), 또는 2,000ft의 분리(산악 지역)가 적용되며, 적절한 레이더 송/수신 보장, 해당 관제 공역의 최저점으로부터 최소 300ft의 분리 등의 조건을 만족하는 고도이다.
- 항공기 A는 표준 계기 출발 절차(SID: Standard Instrument Departure)를 수행 중이었으며, 보고자는 접근 관제 업무 한정을 취득하고자 하는 관제 훈련생을 대상으로 뒤에서 모니터하며 관제 업무를 수행 중이었음. 항공기 A에 대한 관제 업무 한정을 취득하고자 하는 관제 훈련생을 대상으로 뒤에서 모니터하며 관제 업무를 수행 중이었음. 항공기 A에 대한 관제권을 인수하기 전, 협조석* 관제사로부터 출발 경로, SID 및 비행 계획에 관한 정보를 전달받아 해당 항공기를 감시 중이었음.
* 협조석 관제사(Coordination 또는 Flight Data 관제사) : 타 관제 기관 또는 동일 관제 기관 내 좌석 간의 이양/인수 등 협조가 필요한 사항 및 필요한 정보를 전달해주는 역할을 수행하는 관제사
- 최초 교신 당시에 항공기 A의 선회 반경이 예정 경로보다 작았고(통상적인 항공기보다 속도가 느려서), 이에 따라 출발 절차에 규정된 첫 번째 고도 제한(5,000ft 이상 7,000ft 이하)을 간신히 통과할 것으로 판단하였고, 두 번째 고도 제한(9,000ft 이상) 준수는 불가능할 것으로 판단하였음. 또한 항공기와 인접한 MVA가 7,000ft 이상으로 구성되어, 해당 영역에 진입 시 레이더 유도를 통한 항공기 안전을 담보할 수 없는 상황이었음(MVA 미만의 고도에 항공기가 위치하므로 레이더 유도 불가).
- 또한 00비행장에 설정된 다른 계기 출발 절차와는 상이하게, 목적지 공항 방향이 아닌 고고도 장애물 방향으로 경로가 설정되어 교통 흐름이 효율적이지 못했고, 해당 항공기가 저속이기 때문에 다른 항공기의 IFR 운항에 제한을 줄 수 있었음.
- 이에 따라 고고도 장애물 방향으로 진행하던 표준 계기 출발 절차를 취소하고, MVA가 낮은 북쪽 공역으로 레이더 유도를 하도록 관제 훈련생에게 지시하였음. 이때 항공기 A가 위치한 공역의 MVA(5,000ft)가 아닌, 북쪽의 인접한 공역의 MVA(약 3,000ft)에 해당한다고 착각하여, 항공기의 현 고도가 MVA 미만임에도 불구하고 레이더 유도를 실시하여 최저 유도 고도 미만에서의 레이더 유도가 이루어짐.
- (보고자 의견) 최근 관할 TMA(Terminal Control Area, 접근 관제 구역) 내 MVA가 전반적으로 상향되었으나, 인근 공항(비행장)에서 사용 중인 비행 절차는 이를 반영하지 못한 절차가 다수 있다고 사료되어, 관계 기관 간의 논의를 통해 항공교통 안전 및 원활한 교통 흐름이 보장되는 계기 절차의 개정이 이루어지길 바람."""

In [41]:
inference(doc_text)

[]

In [42]:
doc_text = """- ○○비행장에서 이륙을 위하여 활주로 정지선 전에 대기한 후 관제탑에 이륙 허가를 요청하였음.
- 관제사로부터 이륙 허가를 받고 난 직후, 활주로에 진입하기 위한 절차를 수행하며 Final leg(비행장에 시계비행으로 접근하기 위한 사각형의 비행 장주 중, 착륙하기 위해 활주로에 정대하는 최종 구간)를 육안으로 확인하던 중, 약 0.5NM에 접근하는 항공기를 확인하여 관제사에게 이를 보고하였음.
- 이를 보고하자, 관제사는 즉시 이륙 허가를 취소했고, 접근하던 항공기는 이 교신을 듣고 직접 복행하였음."""

In [43]:
inference(doc_text)

[]

In [44]:
doc_text = """항공편은 만석이었습니다. 객실 승무원은 세 명으로, 그중 한 명은 신입이었습니다. 해당 항공편은 이미 90분가량 지연된 상황이었습니다. 좌석 위 선반은 금방 차 버려, 승객들이 안내에 따라 가방을 수납하도록 여러 차례 기내 방송을 했습니다. 
탑승이 끝났을 때 여행 가방 5개가량이 수납되지 못한 상황이었기 때문에, 신입 승무원에게 객실에 수납공간을 만들어 보라고 말했습니다. 나머지 두 명은 출입문을 지켜야 했습니다.
지상 근무 직원들은 출입구 옆에 서서 제가 수화물 반출 여부를 결정할 때까지 기다렸습니다. 하지만 저는 개방된 출입구 앞 위치를 지켜야 했기 때문에, 서 있는 자리에서 최선의 추측을 해야 했습니다. 만약 제가 도어를 오픈 상태로 유지하라 했다가 짐을 내리지 않게 되면 항공기 지연의 책임은 제가 지게 되며, 이에 대한 “관리”를 받게 됩니다. 이렇듯 지연을 발생시키지 말라는 항공사의 압력이 은근히 존재합니다. 결국 저는 직원들에게 출입문을 닫으라고 지시했습니다. 
객실 승무원 세 명 모두가 객실에서 기내 수화물 정리에 매달리느라 아무도 출입구를 지키지 못한 때도 있었습니다. 저희는 통로에 있던 짐을 모두 수납하는 데 간신히 성공했습니다. 
안전 수칙 시범을 끝내고 이륙 준비를 시작하던 중, 승객이 다리 뒤에 숨기고 있던 바퀴 달린 여행 가방 하나와 비상 열의 승객들이 다리 뒤에 두었던 대형 배낭 두 개를 발견했습니다. 여행 가방은 완전히 포화 상태가 된 옷장에, 배낭 두 개는 좌석 위 선반 안에 가까스로 밀어 넣었습니다. """

In [45]:
inference(doc_text)

[]

## (7) 모델 평가

testset 불러오기

In [46]:
test_df = pd.read_csv("../data/rawdata/GYRO_testset.csv")

In [47]:
predicted_keyphrases_list = []
for i in tqdm(range(len(test_df))):
    doc = test_df["본문"][i]
    predicted_keyphrase = inference(doc)
    predicted_keyphrases_list.append('"' + '", "'.join(predicted_keyphrase) + '"')

100% 219/219 [52:44<00:00, 14.45s/it] 


In [48]:
test_df["SBERT 예측 키워드"] = predicted_keyphrases_list

In [49]:
test_df.to_csv("../data/prediction/SBERT.csv", index=False)

In [50]:
exit()